***GENERATED CODE FOR brazilianexports1 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CountryName', 'transformation_label': 'String Indexer'}], 'feature': 'CountryName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '4199', 'mean': '', 'stddev': '', 'min': 'Algeria', 'max': 'Zimbabwe', 'missing': '0', 'distinct': '139'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CountryName'}, {'feature_label': 'CountryName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CountryName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProductName', 'transformation_label': 'String Indexer'}], 'feature': 'ProductName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4199', 'mean': '', 'stddev': '', 'min': 'Albuminoidal substances; modified starches; glues; enzymes', 'max': 'Zinc and articles thereof', 'missing': '0', 'distinct': '85'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProductName'}, {'feature_label': 'ProductName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProductName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'IndustryName', 'transformation_label': 'String Indexer'}], 'feature': 'IndustryName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4199', 'mean': '', 'stddev': '', 'min': 'Automobiles', 'max': 'Textiles, Apparel & Luxury Goods', 'missing': '0', 'distinct': '22'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'IndustryName'}, {'feature_label': 'IndustryName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('IndustryName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SectorName', 'transformation_label': 'String Indexer'}], 'feature': 'SectorName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4199', 'mean': '', 'stddev': '', 'min': 'Communication Services', 'max': 'Materials', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SectorName'}, {'feature_label': 'SectorName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SectorName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4199', 'mean': '', 'stddev': '', 'min': 'Acre', 'max': 'Tocantins', 'missing': '0', 'distinct': '24'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State Abbv', 'transformation_label': 'String Indexer'}], 'feature': 'State Abbv', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4199', 'mean': '', 'stddev': '', 'min': 'AC', 'max': 'TO', 'missing': '0', 'distinct': '24'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State Abbv'}, {'feature_label': 'State Abbv', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State Abbv')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://backendmaster:7077').getOrCreate()
#%run brazilianexports1Hooks.ipynb
try:
	#sourcePreExecutionHook()

	brazilianexports = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Brazilian Exports 7000.csv', 'filename': 'Brazilian Exports 7000.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.61.134', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(brazilianexports)

except Exception as ex: 
	logging.error(ex)
spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run brazilianexports1Hooks.ipynb
try:
	#transformationPreExecutionHook()

	brazilianexportsreg = TransformationMain.run(brazilianexports,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4199", "mean": "2019.05", "stddev": "0.79", "min": "2018", "max": "2020", "missing": "0"}, "updatedLabel": "Year"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4199", "mean": "5.78", "stddev": "3.47", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "Month"}, {"transformationsData": [{"feature_label": "CountryName", "transformation_label": "String Indexer"}], "feature": "CountryName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4199", "mean": "", "stddev": "", "min": "Algeria", "max": "Zimbabwe", "missing": "0", "distinct": "139"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CountryName"}, {"transformationsData": [{"feature_label": "ProductName", "transformation_label": "String Indexer"}], "feature": "ProductName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4199", "mean": "", "stddev": "", "min": "Albuminoidal substances; modified starches; glues; enzymes", "max": "Zinc and articles thereof", "missing": "0", "distinct": "85"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProductName"}, {"transformationsData": [{"feature_label": "IndustryName", "transformation_label": "String Indexer"}], "feature": "IndustryName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4199", "mean": "", "stddev": "", "min": "Automobiles", "max": "Textiles, Apparel & Luxury Goods", "missing": "0", "distinct": "22"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "IndustryName"}, {"transformationsData": [{"feature_label": "SectorName", "transformation_label": "String Indexer"}], "feature": "SectorName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4199", "mean": "", "stddev": "", "min": "Communication Services", "max": "Materials", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SectorName"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4199", "mean": "", "stddev": "", "min": "Acre", "max": "Tocantins", "missing": "0", "distinct": "24"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"feature_label": "State Abbv", "transformation_label": "String Indexer"}], "feature": "State Abbv", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4199", "mean": "", "stddev": "", "min": "AC", "max": "TO", "missing": "0", "distinct": "24"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State Abbv"}]}))

	#transformationPostExecutionHook(brazilianexportsreg)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run brazilianexports1Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(brazilianexportsreg, ["Year", "Month", "CountryName_stringindexer", "IndustryName_stringindexer", "SectorName_stringindexer", "State_stringindexer", "State Abbv_stringindexer"], "ProductName_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

